## Variaciones sobre el modelo inicial 

Vamos a implementar alguna modificación sobre el modelo inicial propuesto en la primera sesión.

Para empezar vamos a definir un conjunto de validación. Con este conjunto de calidación extraído del propio conjunto de entrenamiento estimaremos los mejores parámetros. Dejaremos el conjunto de test sólo para estimar el acierto final con estos mejores parámetros.

Primero leemos los datos y normalizamos:

In [1]:
from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

2023-10-11 22:49:43.306284: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-11 22:49:43.344662: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-11 22:49:43.509323: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-11 22:49:43.509355: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-11 22:49:43.510441: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

training set (60000, 28, 28)
test set (10000, 28, 28)


## Partición training/validación

Nos quedaremos con un 80% de los datos para entrenar y un 20% para validar:

In [2]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

training set (48000, 784)
val set (12000, 784)


## Experimentación con conjunto de validación

Cualquier parámetro a modificar en nuestro modelo debería ser probado sobre el conjunto de validación y escoger aquel que produjera el mejor resultado para probar ese (y sólo ese) sobre el conjunto de test.

In [3]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD

batch_size=128
epochs=5  

hdim=[128,256,512,1024]
best_acc=0.0
for h in hdim:
    model = Sequential()

    model.add(Input(784))
    model.add(Dense(h, activation='relu')) # <-- repetir esta línea tantas veces como número de capas ocultas se quiera
    model.add(Dense(num_classes, activation='softmax'))

    sgd=SGD(learning_rate=0.01, momentum=0.9)


    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
 

    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val)) ## <--- OJO validation set
    
    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        besth=h

print("=============================")
print("Best acc",best_acc)
print("Best hidden dim",besth)
print("=============================")


## PROBAR EL MEJOR CON TEST
model = Sequential()

model.add(Input(784))
model.add(Dense(besth, activation='relu'))  
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))  




Epoch 1/5
375/375 [==============================] - 1s 2ms/step - loss: 0.5401 - accuracy: 0.8516 - val_loss: 0.3158 - val_accuracy: 0.9123
Epoch 2/5
375/375 [==============================] - 1s 1ms/step - loss: 0.2854 - accuracy: 0.9193 - val_loss: 0.2567 - val_accuracy: 0.9273
Epoch 3/5
375/375 [==============================] - 1s 2ms/step - loss: 0.2366 - accuracy: 0.9336 - val_loss: 0.2231 - val_accuracy: 0.9368
Epoch 4/5
375/375 [==============================] - 1s 1ms/step - loss: 0.2034 - accuracy: 0.9424 - val_loss: 0.1977 - val_accuracy: 0.9440
Epoch 5/5
375/375 [==============================] - 1s 1ms/step - loss: 0.1789 - accuracy: 0.9496 - val_loss: 0.1767 - val_accuracy: 0.9513
Epoch 1/5
375/375 [==============================] - 1s 2ms/step - loss: 0.5235 - accuracy: 0.8560 - val_loss: 0.3005 - val_accuracy: 0.9166
Epoch 2/5
375/375 [==============================] - 1s 2ms/step - loss: 0.2693 - accuracy: 0.9243 - val_loss: 0.2407 - val_accuracy: 0.9324
Epoch 3/5
375

## EARLY STOPPING

De la multitud de parámetros a decidir en la definición y entrenamiento de las redes neuronales hay uno que podríamos automatizar. Es el número de epochs a emplear. Dado que tenemos un conjunto de validación vamos a emplearlo para monitorizar cómo evoluciona la convergencia. Si en algún momento el descenso por gradiente parece haber alcanzado una zona de meseta sobre dicho conjunto de validación podremos detener el entrenamiento.

In [4]:
model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback])  



Epoch 1/100
375/375 [==============================] - 2s 5ms/step - loss: 0.4909 - accuracy: 0.8679 - val_loss: 0.2930 - val_accuracy: 0.9178
Epoch 2/100
375/375 [==============================] - 2s 4ms/step - loss: 0.2615 - accuracy: 0.9260 - val_loss: 0.2326 - val_accuracy: 0.9369
Epoch 3/100
375/375 [==============================] - 2s 4ms/step - loss: 0.2112 - accuracy: 0.9417 - val_loss: 0.1966 - val_accuracy: 0.9474
Epoch 4/100
375/375 [==============================] - 2s 4ms/step - loss: 0.1770 - accuracy: 0.9511 - val_loss: 0.1732 - val_accuracy: 0.9527
Epoch 5/100
375/375 [==============================] - 2s 4ms/step - loss: 0.1528 - accuracy: 0.9573 - val_loss: 0.1563 - val_accuracy: 0.9578
Epoch 6/100
375/375 [==============================] - 2s 4ms/step - loss: 0.1347 - accuracy: 0.9635 - val_loss: 0.1418 - val_accuracy: 0.9603
Epoch 7/100
375/375 [==============================] - 2s 4ms/step - loss: 0.1199 - accuracy: 0.9671 - val_loss: 0.1316 - val_accuracy: 0.9633

## **Ejercicio**:

Entrenar una red neuronal MLP para el problema MNIST. Los valores que se quieren probar son los siguientes:

-   Número de capas ocultas [1,2,3]
-   Dimensión de las capas ocultas [512,1024]
-   Learning rate [0.01, 0.001, 0.0001]
-   Batch_size [128]
-   Epochs con early_stopping

Obtener la mejor combinación con el conjunto de validación y probarla finalmente sobre el conjunto de test

In [6]:
model = Sequential()

model.add(Input(784))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

learning_rates = [0.01, 0.001, 0.0001]
batch_size = 128

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

for lr in learning_rates:
    sgd=SGD(learning_rate=lr, momentum=0.9)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_val, y_val),
                        callbacks=[callback])
    
    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        besth=h

print("=============================")
print("Best acc",best_acc)
print("Best hidden dim",besth)
print("=============================")

Epoch 1/100
375/375 [==============================] - 2s 6ms/step - loss: 0.5041 - accuracy: 0.8595 - val_loss: 0.2536 - val_accuracy: 0.9298
Epoch 2/100
375/375 [==============================] - 2s 5ms/step - loss: 0.2246 - accuracy: 0.9358 - val_loss: 0.1944 - val_accuracy: 0.9437
Epoch 3/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1678 - accuracy: 0.9526 - val_loss: 0.1647 - val_accuracy: 0.9515
Epoch 4/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1317 - accuracy: 0.9631 - val_loss: 0.1406 - val_accuracy: 0.9592
Epoch 5/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1090 - accuracy: 0.9688 - val_loss: 0.1190 - val_accuracy: 0.9663
Epoch 6/100
375/375 [==============================] - 2s 5ms/step - loss: 0.0907 - accuracy: 0.9745 - val_loss: 0.1077 - val_accuracy: 0.9688
Epoch 7/100
375/375 [==============================] - 2s 6ms/step - loss: 0.0775 - accuracy: 0.9780 - val_loss: 0.0999 - val_accuracy: 0.9697